### ML notebook

In [6]:
n_samples = 1000
gens_per_ideal = 2

In [7]:
# P.<x,y> = PolynomialRing(QQ, 2, order='deglex')
P.<x,y> = PolynomialRing(QQ, 2, order='lex')
# P.<x,y> = PolynomialRing(QQ, 2, order='degrevlex')
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('wdegrevlex',(1,3)))
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('negwdegrevlex',(1,3)))

#### Functions to create features and label

In [8]:
def is_GB_worth(GB): # Receives a GB
    n_depend1=0
    n_depend2=0
    for pol in GB:
        depend_x=False
        depend_y=False
        
        # Comprobamos si depende de x y de y
        if pol.degree(x) > 0:
            depend_x=True
        if pol.degree(y) > 0:
            depend_y=True
            
        # Actualizamos el numero de de polinomios que dependen de cada cosa
        if depend_x and depend_y:
            n_depend2 += 1
        elif depend_x or depend_y:
            n_depend1 += 1
    if n_depend1 >= n_depend2:
        return True
    else:
        return False
    
def terms_on_x(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[0] > 0:
                count+=1
    return count
        
def terms_on_y(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[1] > 0:
                count+=1
    return count

In [9]:
from collections import defaultdict

def n_homogeneous_comps(pol):
    dic = defaultdict(pol.parent())
    
    for coeff,monom in pol:
        dic[monom.degree()] += coeff * monom
        
    return len(dic)

#### Voy a generar una lista de ideales para los polinomios generados aleatoriamente y con información adicional

In [10]:
lstOfIdeals = []

for i in range(n_samples):
    generators = []
    n_homogeneous = 0
    total_homogeneous_comps=0
    
    for j in range(gens_per_ideal):
        p = P.random_element(degree=2)
        
        # homogeneous components for the given polynomial
        total_homogeneous_comps += n_homogeneous_comps(p)
        
        generators.append(p)
        if p.is_homogeneous():
            n_homogeneous += 1
    
    # Definimos el ideal a partir de los generadores anteriores
    I = ideal(generators)
    B = I.groebner_basis()
    lstOfIdeals.append([I, 
                        n_homogeneous, 
                        terms_on_x(generators), 
                        terms_on_y(generators), 
                        total_homogeneous_comps, 
                        is_GB_worth(B)])

# Conjunto de polinomios generado de manera aleatoria
lstOfIdeals[:3]

[[Ideal (4*x*y + 30*y - 1/4, -x^2 + x*y + 2*y^2 + 6*y - 1/3) of Multivariate Polynomial Ring in x, y over Rational Field,
  0,
  3,
  5,
  6,
  True],
 [Ideal (-2/3*x - 1/2*y^2 - 2*y, -x^2 - 1/6*x + 1/3*y^2 - 2) of Multivariate Polynomial Ring in x, y over Rational Field,
  0,
  3,
  3,
  5,
  True],
 [Ideal (41/2*x^2 - 1/2*x - 1/17*y^2 + 1, -2*x*y - 10*x - y^2 - 17/2*y) of Multivariate Polynomial Ring in x, y over Rational Field,
  0,
  4,
  4,
  5,
  True]]

In [11]:
!sage --pip install numpy scipy sklearn pandas

#### Definimos las features

In [12]:
import numpy as np

X = np.array([[item[1],item[2],item[3],item[4]] for item in lstOfIdeals])
y = np.array([item[5] for item in lstOfIdeals])

In [13]:
X[:5]

array([[0, 3, 5, 6],
       [0, 3, 3, 5],
       [0, 4, 4, 5],
       [0, 5, 4, 4],
       [0, 3, 5, 5]])

In [14]:
y[:5]

array([ True,  True,  True,  True,  True])

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
from sklearn import svm

classifier = svm.SVC()

In [17]:
classifier.fit(X_train, y_train)

/home/santi/SageMath/local/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [18]:
preds = classifier.predict(X_test)

In [19]:
hits=0
fails=0
for i in range(len(y_test)):
    if y_test[i] == preds[i]:
        hits+=1
    else:
        fails+=1

In [20]:
print('Hits: ' + str(hits), 'Fails: '+str(fails))

('Hits: 320', 'Fails: 10')
